<a href="https://colab.research.google.com/github/NickAldewereld/zoekwoordtool/blob/main/Keywords_clusteren_SERP_simularity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hallo daar! 👋

Mijn naam is Frank, SEO specialist bij [Maatwerk Online](https://www.maatwerkonline.nl/). Sinds begin dit jaar ben ik begonnen met SEO processen automatiseren om zo effectiever te zijn in mijn dagelijkse werk. Eén van deze projecten is het automatisch clusteren van keywords middels Python. Via deze Google Colab deel ik mijn code met jou.

Middels dit script kun je keywords clusteren op basis van SERP simularity. Als van de top tien organische resultaten een bepaald aantal links overeenkomt (standaard staat dit op 3) dan clustert hij deze onder één onderwerp. Op deze manier kun je datagedreven te werk gaan en bespaar je tijd.

<b>Werking van de code</b>

*   Keywords ophalen uit het Excel bestand
*   Zoekresultaten ophalen op basis van de keywords uit het bestand
*   Links van de top 10 organische resultaten hieruit halen
*   Keywords met een overlap van 3 dezelfde links samenvoegen
*   Samengevoegde keywords als Excel downloaden

In de eerste stap upload je jouw Excel bestand. In de tweede stap gaat het script de keywords voor je clusteren. Let wel op dat je zelf bij stap 2 een API key van [SerpAPI](https://serpapi.com/) key toevoegd.

Heb je vragen of verbeteringen, neem dan contact met me op via frankvandijk@maatwerkonline.nl (ik ben relatief nieuw in de Python wereld). Als je op de hoogte wilt blijven van waar ik mee bezig ben kan je me volgen op [LinkedIn](https://www.linkedin.com/in/frankvndijk/) en [X (Twitter)](https://x.com/frankvndijk).

<h2>Stap 1: Excel uploaden</h2>
Door de code hieronder uit te voeren kan je jouw Excel bestand uploaden. Let op dat alle keywords in kolom A staan. De cel A1 wordt niet meegenomen.

In [ ]:
from google.colab import files
import io
import pandas as pd
import requests

uploaded = files.upload()

for fn in uploaded.keys():
  print('Je hebt een bestand geüpload'.format(
      name=fn, length=len(uploaded[fn])))

<h2>Stap 2: Keywords clusteren</h2>
Door de code hieronder uit te voeren worden je keywords geclusterd. Voeg zelf de API key van SerpAPI toe en houdt er rekening mee dat hun API ongeveer 1,5 seconden per keywords nodig heeft.


In [ ]:
API_KEY_SERPAPI = #plaats hier de API key van SerpAPI als string

def getting_keywords():
    excel_filename = next(iter(uploaded))
    df = pd.read_excel(excel_filename)
    cell_values_as_strings = df.astype(str).values.flatten().tolist()
    return cell_values_as_strings


def scraping_google():
    serp_data = []
    for i in KEYWORDS:
        params = {
            "q": i,
            "google_domain": "google.nl",
            "hl": "nl",
            "gl": "nl",
            "api_key": API_KEY_SERPAPI,
        }
        response = requests.get(url="https://serpapi.com/search", params=params)
        data = response.json()
        serp_data.append(data)
    return serp_data


def get_organic_results(serp_data):
    full_serp = {}
    ammount = 0
    for i in serp_data:
        current_serp = serp_data[ammount]
        ammount += 1
        keyword = current_serp["search_parameters"]["q"]
        rankings = []
        for position in range(0, 9):
            if position >= len(current_serp["organic_results"]):
                break
            rankings.append(current_serp["organic_results"][position]["link"])
        full_serp[keyword] = rankings
    return full_serp


def clustering_keywords(dict):
    clustered_keywords = {}
    keyword_searched = set()

    for current_keyword, current_url in dict.items():
        if current_keyword in keyword_searched:
            continue
        keyword_searched.add(current_keyword)
        clustered_keywords[current_keyword] = [current_keyword]
        for compared_keyword, compared_url in dict.items():
            if compared_keyword == current_keyword or compared_keyword in keyword_searched:
                continue
            agreements = len(set(current_url) & set(compared_url))

            if agreements >= 3:
                clustered_keywords[current_keyword].append(compared_keyword)
                keyword_searched.add(compared_keyword)
    return clustered_keywords


def clustered_keywords_to_excel(clustered_keywords, filename='Keywords.xlsx'):
    df = pd.DataFrame()
    for key, items in clustered_keywords.items():
        temp_df = pd.DataFrame({
            'Cluster': [key] * len(items),
            'Keywords': items
        })
        df = pd.concat([df, temp_df], ignore_index=True)
    df.to_excel(filename, index=False)
    files.download(filename)


KEYWORDS = getting_keywords()
serp_data = scraping_google()
organic_results = get_organic_results(serp_data)
clustered_keywords = clustering_keywords(organic_results)
clustered_keywords_to_excel(clustered_keywords)

Vragen of verbeteringen voor het script? Neem contact op met frankvandijk@maatwerkonline.nl.